In [1]:
import re
from Modelos.base_model import base_model
from Modelos.serie_model import serie_model
from Modelos.parallel_model import parallel_model
from Modelos.hibrid_model import hibrid_model
from utils.utils import mostrarResultadosTabla

In [2]:
N = 11
L = 10
c_values = {0: 5, 1: 12, 2: 20}
confiabilidad_values = {0: 0.9, 1: 0.95, 2: 0.99}
modeloBase = base_model(N, c_values)

Restricted license - for non-production use only - expires 2026-11-23


In [3]:
# Crear y optimizar el modelo en serie
try:
    costoSerie, variablesDecisionSerie, modeloSerie = serie_model(modeloBase, N, L)
    mostrarResultadosTabla(N, costoSerie, variablesDecisionSerie)
except Exception as e:
    print(f"Error: {e}")

Cantidad de Nodos: 11
Resultado de la Optimización:
Costo Total: 155.0
Costo nodos: 55.0
Costo enlaces: 100.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0
11         1         0          0


In [4]:
# Crear y optimizar el modelo en paralelo
try:
    costoParalelo, variablesDecisionParalelo, modeloParalelo = parallel_model(modeloBase, N, L)
    mostrarResultadosTabla(N, costoParalelo, variablesDecisionParalelo)
except Exception as e:
    print(f"Error: {e}")

Cantidad de Nodos: 11
Resultado de la Optimización:
Costo Total: 605.0
Costo nodos: 55.0
Costo enlaces: 550.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0
11         1         0          0


In [5]:
# Crear y optimizar el modelo híbrido
try:
    costoHibrido, variablesDecisionHibrido, modeloHibrido = hibrid_model(modeloBase, N, L)
    mostrarResultadosTabla(N, costoHibrido, variablesDecisionHibrido, "hibrido")
except Exception as e:
    print(f"Error: {e}")

Cantidad de Nodos: 11
Resultado de la Optimización:
Costo Total: 165.0
Costo nodos: 55.0
Costo enlaces: 110.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0
11         1         0          0
Nodos activos (y):
    Subred 0  Subred 1  Subred 2  Subred 3
1          0         0         1         0
2          1         0         0         0
3          0         0         1         0
4          1         0         0         0
5          0         0         1         0
6          1         0         0         0
7          1         0         0         0
8          1         0         0         0
9          1         0         0         0
10         1  